In [18]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

In [19]:
# import data & add daily returns column
def import_data(stocks):
    stocks_data = {}
    for stock in stocks:
        stock_data = yf.download(stock, start='2010-01-01', end='2025-01-01')
        stock_data['Return'] = stock_data['Close'].pct_change()
        stocks_data[stock] = stock_data
    return stocks_data

#### **1. Calculate Annual Expected Return of an Asset**:

The annual expected return for an asset i is calculated as follows _(assuming 252 trading days)_:
$$E(R_{annual}) = (1 + E(R_{daily}))^{252} - 1$$
where,
$$E(R_{daily}) = \frac{\displaystyle\sum R_t}{n}$$
and $R_t$ is the return on day $t$ (daily return).


In [20]:
# calculate annual expected return
def annual_expected_return(data):
    results_dict = {}
    # calculate expected daily return
    returns_data = pd.DataFrame({stock: data[stock]['Return'] for stock in data.keys()})
    e_daily_returns = np.sum(returns_data, axis=0) / len(returns_data)
    # calculate expected annual return
    e_annual_returns = (1 + e_daily_returns)**252 - 1
    # record results
    results_dict['Daily Returns'] = e_daily_returns
    results_dict['Annual Returns'] = e_annual_returns

    return results_dict

#### **2. Calculate Expected Portfolio Return**:

The expected portfolio return is calculated as:
$$E(R_p) = \displaystyle\sum (\omega_i \times E(R_i))$$
where:
- $E(R_p)$ = Expected return of the portfolio (in 1 year)
- $\omega_i$ = Weight (proportion) of asset i in the portfolio
- $E (R_i)$ = Expected return of asset i
- $n$ = Number of assets in the portfolio

In [21]:
def portfolio_return(data, weights):
    # extract expected annual returns
    e_annual = annual_expected_return(data)
    e_annual_returns = np.array(e_annual['Annual Returns'].values)
    # compute expected portfolio return
    return sum(weights[i] * e_annual_returns[i] for i in range(len(weights)))

#### **3. Calculate Volatility**:

The annualized volatility for an asset i is calculated as follows _(assuming 252 trading days)_:
$$\sigma_{annual} = \sigma_{daily} \times \sqrt{252}$$
where $\sigma_{daily}$ is defined as:
$$\sigma_{daily} = \sqrt{\frac{\displaystyle\sum (R_t - \hat{R})^2}{n-1}}$$
where:
- $R_t$: Daily return on day $t$
- $\hat{R}$: Average daily returns
- $n$: Number of trading days

Portfolio volatility $\sigma_p$ is calculated as follows:
$$\sigma_p = \sqrt{\omega^T \gamma \omega}$$
where:
- $\omega$ = Portfolio weights (vector of size n)
- $\gamma$ = Annualized covariace matrix (constructed using annualized volatilities and correlations)
- $\omega^T$ = Transpose of weights vector

In [22]:
# calculate annualized volatility
def annualized_volatility(data):
    results_dict = {}
    # compute average daily returns
    returns_data = pd.DataFrame({stock: data[stock]['Return'] for stock in data.keys()})
    # compute standard deviation of daily returns
    volatility_daily = returns_data.std()
    # compute annualized volatility
    volatility_annualized = volatility_daily * np.sqrt(252)
    # record results
    results_dict['Daily Volatility'] = volatility_daily
    results_dict['Anual Volatility'] = volatility_annualized

    return results_dict

In [23]:
# calculate portfolio volatility
def portfolio_volatility(data, weights):
    returns_data = pd.DataFrame({stock: data[stock]['Return'] for stock in data.keys()})
    returns_data.dropna(inplace=True)
    # compute covariance matrix
    daily_cov = np.cov(returns_data, rowvar=False)
    annualized_cov = daily_cov * 252
    # convert weights & volatilities to arrays
    w = np.array(weights)
    wT = np.transpose(w)
    # compute portfolio volatility
    return np.sqrt(np.dot(wT, np.dot(annualized_cov, w)))

#### **4. Sharpe Ratio:**
$$\text{Sharpe Ratio} = \frac{\text{Portfolio Return - Risk Free Rate}}{\text{Portfolio Volatility}}

In [24]:
def sharpe_ratio(data, weights):
    results_dict = {}
    p_return = portfolio_return(data, weights)
    rfr = 0.04 # 4% (U.S. Treasury Yield)
    p_volatility = portfolio_volatility(data, weights)
    # compute sharpe ratio
    return (p_return - rfr) / p_volatility

#### **5. Gradient of the Sharpe Ratio**:

We wish to find the gradient of the sharpe ratio formula.

**Sharpe Ratio Formula**:
$$\text{Sharpe Ratio} = \frac{E(R_p) - R_f}{\sigma_p}$$

where:
- $E(R_p) = \bold{w^T}\mu$ (Expected Portfolio Return)
- $R_f =$ risk-free rate (assumed to be 0.04)
- $\sigma_p = \sqrt{\bold{w^T}\gamma\bold{w}}$ (Portfolio Volatility)
- $\bold{w} =$ Vector of portfolio weights
- $\mu =$ Vector of expected returns
- $\gamma =$ Covariance matrix of stock returns

We begin by computing partial derivatives.

**Gradient of Portfolio Return $E(R_p)$**:
$$\frac{\partial E(R_p)}{\partial\bold{w}} = \mu$$

**Gradient of Portfolio Volatility $\sigma_p$**:
$$\frac{\partial\sigma_p}{\partial\bold{w}} = \frac{\gamma\bold{w}}{\sigma_p}$$

Thus we have,

**Gradient of the Sharpe Ratio**:
$$\nabla SR = \frac{\frac{\partial E(R_p)}{\partial\bold{w}}\cdot\sigma_p - E(R_p)\cdot\frac{\partial\sigma_p}{\partial\bold{w}}}{\sigma_p^2} = \frac{\mu\sigma_p - E(R_p)\cdot\frac{\gamma\bold{w}}{\sigma_p}}{\sigma_p^2}$$

In [25]:
def sharpe_ratio_gradient(data, weights):
    p_return = portfolio_return(data, weights)
    p_volatility = portfolio_volatility(data, weights)
    # compute mu (vector of expected returns)
    mu = np.array(annual_expected_return(data)['Annual Returns'].values)
    # compute gamma (covariance matrix of stock returns)
    returns_data = pd.DataFrame({stock: data[stock]['Return'] for stock in data.keys()})
    returns_data.dropna(inplace=True)
    daily_cov = np.cov(returns_data, rowvar=False)
    gamma = daily_cov * 252
    # compute gradient
    dE = mu
    dsigma = np.dot(gamma, weights) / p_volatility
    grad_sharpe = (dE * p_volatility - p_return * dsigma) / (p_volatility)**2

    return grad_sharpe

#### **6. Gradient Ascent**:

For portfolio optimization, we use a **gradient ascent** algorithm to maximize the **Sharpe Ratio** by iteratively adjusting asset allocations.

1. **Gradient Calculation**: The algorithm computes the gradient at the current `weights` position, indicating the direction of the steepest increase in Sharpe Ratio.
2. **Regularization**: To encourage portfolio diversification, an L2 regularization term is applied to the gradient. Without this, the algorithm may exploit the Sharpe Ratio formula by concentrating allocations in high-return assets while disregarding lower-risk investments.
3. **Weight Update**: The portfolio weights are adjusted in the direction of the gradient to improve the Sharpe Ratio.
4. **Projection**: The updated weights are projected onto the simplex to ensure all values remain **non-negative** and **sum to 1**, further promoting a well-diversified portfolio.
5. **Iterative Optimization**: Steps 1-4 are repeated for `num_iterations` until weights converge to an optimal Sharpe Ratio.

### **Additional Features**:
- **Step Size Decay**: The learning rate (`alpha`) is reduced by a factor of `decay` after each iteration, improving convergence stability and preventing overshooting.
- **Early Stopping Condition**: If the Sharpe Ratio decreases for **5 consecutive iterations**, the algorithm terminates early to prevent unnecessary computations and potential degradation of results.

In [26]:
# projection to simplex (ensure all weights are non-negative and sum to 1)
def project_to_simplex(weights, epsilon=1e-4, lambda_reg=0.01):
    # Ensure weights have a minimum value to prevent zero allocations
    weights = np.maximum(weights, epsilon)

    # Apply L2 regularization to prevent extreme allocations
    weights -= lambda_reg * weights

    # Sort weights in descending order for projection
    sorted_weights = np.sort(weights)[::-1]
    cumulative_sum = np.cumsum(sorted_weights) - 1  # Adjust for sum constraint

    # Find the largest rho index that satisfies the condition
    rho = np.where(sorted_weights - (cumulative_sum / (np.arange(len(weights)) + 1)) > 0)[0][-1]
    lambda_ = cumulative_sum[rho] / (rho + 1)

    # Project weights onto the simplex while ensuring a minimum threshold
    projected_weights = np.maximum(weights - lambda_, epsilon)

    # Normalize to sum to 1
    return projected_weights / np.sum(projected_weights)

In [38]:
# implement gradient ascent to maximize the Sharpe Ratio
def portfolio_optimization(data, weights, alpha=0.5, decay=0.99, num_iterations=200, lambda_reg=0.05, patience=7):
    recent_sharpe = -np.inf
    counter = 0
    results = {'Iteration': [0], 
               'Sharpe Ratio': [sharpe_ratio(data, weights)],
    }
    for i in range(num_iterations):
        grad_sharpe = sharpe_ratio_gradient(data, weights)
        # apply regularization to avoid extreme weights
        grad_sharpe -= lambda_reg * np.array(weights)
        # update gradient ascent
        weights += alpha * grad_sharpe
        # projection with bias to prevent zero weights
        weights = project_to_simplex(weights)
        # decay
        alpha *= decay
        # print results
        sharpe = sharpe_ratio(data, weights)
        # print(weights)
        # print(sharpe)
        # record results
        results['Iteration'].append(i+1)
        results['Sharpe Ratio'].append(sharpe)
        # check for sharpe ratio decrease
        if sharpe < recent_sharpe:
            counter += 1
            if counter >= patience:
                # print('There was 5 consecutive decreases in Sharpe Ratio, Terminating...')
                break
        else:
            recent_sharpe = sharpe
            counter = 0

    results['Optimal Portfolio'] = pd.Series(weights, index=list(data.keys()))
    display = results['Optimal Portfolio'].map(lambda x: f"{x * 100:.2f}%")

    print('Total Iterations Completed: ' + str(max(results['Iteration'])))
    print('Sharpe Ratio: ' + str(sharpe))
    print('Computed Optimal Portfolio: \n' + str(display))

    return results

### _Display Dashboard_

In [ ]:
def final_analysis(stocks, weights, confidence_level = 0.95):
    data = import_data(stocks)
    returns_data = pd.DataFrame({stock: data[stock]['Return'] for stock in stocks})
    returns_data.dropna(inplace=True)
    metrics = {
        'Metric': ['VaR', 'CVaR', 'Volatility', 'Return', 'Sharpe Ratio'],
        'Value': []
    }
    # optimize portfolio
    optimize = portfolio_optimization(data, weights, alpha=0.5)
    optimized_weights = np.array(optimize['Optimal Portfolio'])
    portfolio_returns = np.dot(returns_data, optimized_weights)
    # plot convergence
    plt.figure(figsize=(8, 4))
    plt.plot(optimize['Iteration'], optimize['Sharpe Ratio'], marker='o', color='blue', markersize='5', label='Sharpe Ratio')

    plt.xlabel('Iteration', fontsize=12)
    plt.ylabel('Sharpe Ratio', fontsize=12)
    plt.title('Sharpe Ratio Convergence Over Iterations', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)

    plt.show()
    # compute VaR
    var = np.percentile(portfolio_returns, 100 * (1 - confidence_level))
    # compute CVaR
    cvar = "{:,.2%}".format(portfolio_returns[portfolio_returns <= var].mean())
    var = "{:,.2%}".format(var)
    # compute portfolio volatility
    p_volatility = "{:,.2%}".format(portfolio_volatility(data, optimized_weights))
    # compute expected portfolio return
    p_return = "{:,.2%}".format(portfolio_return(data, optimized_weights))
    # compute sharpe ratio
    sharpe = round(sharpe_ratio(data, optimized_weights), 2)
    # record results and display
    metrics['Value'].extend([var, cvar, p_volatility, p_return, sharpe])
    display = pd.DataFrame.from_dict(metrics)
    
    return display

In [ ]:
def initial_analysis(stocks, weights, confidence_level = 0.95):
    data = import_data(stocks)
    returns_data = pd.DataFrame({stock: data[stock]['Return'] for stock in stocks})
    returns_data.dropna(inplace=True)
    portfolio_returns = np.dot(returns_data, weights)
    metrics = {
        'Metric': ['VaR', 'CVaR', 'Volatility', 'Return', 'Sharpe Ratio'],
        'Value': []
    }
    # create corrolation matrix
    corr = returns_data.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True)
    plt.title("Stock Returns Correlation Matrix")
    plt.show()
    # compute VaR
    var = np.percentile(portfolio_returns, 100 * (1 - confidence_level))
    # compute CVaR
    cvar = "{:,.2%}".format(portfolio_returns[portfolio_returns <= var].mean())
    var = "{:,.2%}".format(var)
    # compute portfolio volatility
    p_volatility = "{:,.2%}".format(portfolio_volatility(data, weights))
    # compute expected portfolio return
    p_return = "{:,.2%}".format(portfolio_return(data, weights))
    # compute sharpe ratio
    sharpe = round(sharpe_ratio(data, weights), 2)
    # record results and display
    metrics['Value'].extend([var, cvar, p_volatility, p_return, sharpe])
    display = pd.DataFrame.from_dict(metrics)
    
    return display